# SN light curve fitting

This exploratory notebook tries to use tape to fit SN Ia light curves using a SALT3 model from SNCosmo

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sncosmo
from tape import Ensemble
from tape.utils import ColumnMapper
import pandas as pd
import dask.dataframe as dd
from snutils import PanPlusUtils
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
from datautils import fit_salt2,fit_salt2_pp


In [3]:
ens = Ensemble(client=False)

In [4]:
ppsn = PanPlusUtils()
fit_lcparams = ppsn.fit_lcparams

In [5]:
kcorfiles = glob.glob(os.path.expandvars("data/salt3/kcor/*.input"))
dflist = []
for f in kcorfiles:
    df = ppsn._read_snana_kcor_input(f,rename_bands=True)
    dflist.append(df)
filtmap = pd.concat(dflist)
def replace_str(row):
    for col in ['filtname','survey']:
        row[col] = row[col].replace('CSPDR2','CSPDR3')
    return row
filtmap = filtmap.apply(replace_str,axis=1)
filtmap = filtmap.drop_duplicates()
filtmap.head()

,filtname,filtstr,filtfile,refmag,survey,magsys,filtpath
0,CSPDR3-Bessell-U,U,sux-shifted.dat_5A,9.724,CSPDR3,BD17,$SNDATA_ROOT/filters/PS1/Pantheon/SNLS3-Landolt
1,CSPDR3-Bessell-B,B,sb-shifted.dat_5A,9.907,CSPDR3,BD17,$SNDATA_ROOT/filters/PS1/Pantheon/SNLS3-Landolt
2,CSPDR3-Bessell-V,V,sv-shifted.dat_5A,9.464,CSPDR3,BD17,$SNDATA_ROOT/filters/PS1/Pantheon/SNLS3-Landolt
3,CSPDR3-Bessell-R,R,sr-shifted.dat_5A,9.166,CSPDR3,BD17,$SNDATA_ROOT/filters/PS1/Pantheon/SNLS3-Landolt
4,CSPDR3-Bessell-I,I,si-shifted.dat_5A,8.846,CSPDR3,BD17,$SNDATA_ROOT/filters/PS1/Pantheon/SNLS3-Landolt


In [6]:
ppsn.register_bands_for_sncosmo(filtmap=filtmap)

In [7]:
ens = Ensemble()

In [8]:
src,obj = ppsn.get_sn_data('data/salt3/SALT3TRAIN_PanPlus/SALT3TRAIN_K21_Fou*/',rename_bands=True,rename_bands_thres=100)

In [9]:
src.head()

,VARLIST:,MJD,FLT,FIELD,FLUXCAL,FLUXCALERR,MAG,MAGERR,PSF,SNID,FLT_str
0,OBS:,57215.27501,Foundation_DR1-PS1-g,VOID,9615.449,202.204,17.543,0.023,2.39,ASASSN-15mg,
1,OBS:,57221.27233,Foundation_DR1-PS1-g,VOID,9623.916,234.360,17.542,0.026,3.21,ASASSN-15mg,
2,OBS:,57227.27655,Foundation_DR1-PS1-g,VOID,6916.386,189.081,17.900,0.030,1.86,ASASSN-15mg,
3,OBS:,57233.27105,Foundation_DR1-PS1-g,VOID,3707.125,176.033,18.577,0.051,2.08,ASASSN-15mg,
4,OBS:,57242.27707,Foundation_DR1-PS1-g,VOID,1590.206,97.227,19.496,0.066,2.08,ASASSN-15mg,


In [10]:
obj.head()

,SURVEY,SNTYPE,SNID,RA,DEC,MWEBV,FILTERS,SEARCH_PEAKMJD,HOSTGAL_LOGMASS_err,HOSTGAL_LOGMASS,PIXSIZE,REDSHIFT_HELIO_err,REDSHIFT_HELIO,REDSHIFT_CMB_err,REDSHIFT_CMB,VPEC_err,VPEC,SURVEY2
0,FOUNDATION,1.0,ASASSN-15mg,233.094056,41.85038,0.028,griz,57221.27273,0.029,10.134,0.25,0.000012,0.042778,0.000012,0.043071,250.0,-289.7960,Foundation_DR1
0,FOUNDATION,1.0,ASASSN-17bs,268.025540,21.566021,0.076,griz,57794.60710,0.012,10.735,0.25,0.000020,0.020397,0.000020,0.020135,250.0,-125.0670,Foundation_DR1
0,FOUNDATION,1.0,ASASSN-16ay,108.059415,7.239544,0.059,griz,57418.42390,0.001,10.615,0.25,0.000013,0.028343,0.000013,0.028953,250.0,-124.5780,Foundation_DR1
0,FOUNDATION,1.0,AT2017lm,45.384899,-15.0159632,0.059,griz,57776.23572,0.013,10.110,0.25,0.000064,0.030636,0.000064,0.030023,250.0,-83.3033,Foundation_DR1
0,FOUNDATION,1.0,MASTEROTJ15,229.196787,28.628617,0.024,griz,57585.29938,0.100,6.000,0.25,0.000015,0.056436,0.000015,0.056904,250.0,24.7887,Foundation_DR1


In [11]:
col_map = ColumnMapper(id_col="SNID",
    time_col="MJD",
    flux_col="FLUXCAL",
    err_col="FLUXCALERR",
    band_col="FLT")

#### try using pandas df first

In [12]:
ens.from_pandas(src,obj,column_mapper=col_map,npartitions=1,sync_tables=False)

/Users/mi/Work/tape/src/tape/ensemble.py:1294: UserWarning: Divisions for object are not set, certain downstream dask operations may fail as a result. We recommend setting the `sort` or `sorted` flags when loading data to establish division information.
  warnings.warn(
/Users/mi/Work/tape/src/tape/ensemble.py:1294: UserWarning: Divisions for source are not set, certain downstream dask operations may fail as a result. We recommend setting the `sort` or `sorted` flags when loading data to establish division information.
  warnings.warn(


In [13]:
# ens.query("SNID == 'ASASSN-15mg'","object")

In [14]:
obj = ens.compute("object")
obj

,SURVEY,SNTYPE,RA,DEC,MWEBV,FILTERS,SEARCH_PEAKMJD,HOSTGAL_LOGMASS_err,HOSTGAL_LOGMASS,PIXSIZE,REDSHIFT_HELIO_err,REDSHIFT_HELIO,REDSHIFT_CMB_err,REDSHIFT_CMB,VPEC_err,VPEC,SURVEY2
SNID,,,,,,,,,,,,,,,,,
ASASSN-15mg,FOUNDATION,1.0,233.094056,41.8503804,0.028,griz,57221.27273,0.029,10.134,0.25,0.000012,0.042778,0.000012,0.043071,250.0,-289.7960,Foundation_DR1
ASASSN-17bs,FOUNDATION,1.0,268.025540,21.5660211,0.076,griz,57794.60710,0.012,10.735,0.25,0.000020,0.020397,0.000020,0.020135,250.0,-125.0670,Foundation_DR1
ASASSN-16ay,FOUNDATION,1.0,108.059415,7.2395438,0.059,griz,57418.42390,0.001,10.615,0.25,0.000013,0.028343,0.000013,0.028953,250.0,-124.5780,Foundation_DR1
AT2017lm,FOUNDATION,1.0,45.384899,-15.0159632,0.059,griz,57776.23572,0.013,10.110,0.25,0.000064,0.030636,0.000064,0.030023,250.0,-83.3033,Foundation_DR1
MASTEROTJ15,FOUNDATION,1.0,229.196787,28.6286166,0.024,griz,57585.29938,0.100,6.000,0.25,0.000015,0.056436,0.000015,0.056904,250.0,24.7887,Foundation_DR1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN2016hhv,FOUNDATION,1.0,353.952050,20.7088099,0.051,griz,57696.21158,0.002,11.496,0.25,0.000140,0.061940,0.000140,0.060675,250.0,-151.9651,Foundation_DR1
AT2017oz,FOUNDATION,1.0,211.673083,30.3188275,0.011,griz,57787.65318,0.014,10.036,0.25,0.000010,0.055689,0.000010,0.056413,250.0,78.8558,Foundation_DR1
AT2016hns,FOUNDATION,1.0,324.921046,24.4064909,0.063,griz,57698.26186,0.017,10.609,0.25,0.000060,0.038170,0.000060,0.037040,250.0,-105.3123,Foundation_DR1


In [15]:
src = ens.compute("source")
src

,VARLIST:,MJD,FLT,FIELD,FLUXCAL,FLUXCALERR,MAG,MAGERR,PSF,FLT_str
SNID,,,,,,,,,,
ASASSN-15mg,OBS:,57215.27501,Foundation_DR1-PS1-g,VOID,9615.449,202.204,17.543,0.023,2.39,
PS15adh,OBS:,57136.34640,Foundation_DR1-PS1-g,VOID,2545.605,103.627,18.986,0.044,1.84,
PS15bzz,OBS:,57284.50289,Foundation_DR1-PS1-g,VOID,3873.670,120.040,18.530,0.034,1.87,
AT2016gsu,OBS:,57669.35730,Foundation_DR1-PS1-g,VOID,3732.409,117.684,18.570,0.034,1.75,
ASASSN-15bc,OBS:,57042.21948,Foundation_DR1-PS1-g,VOID,12357.885,181.802,17.270,0.016,1.88,
...,...,...,...,...,...,...,...,...,...,...
Gaia16aki,OBS:,57541.44376,Foundation_DR1-PS1-z,VOID,537.330,133.651,20.674,0.269,1.46,
Gaia16aki,OBS:,57786.63516,Foundation_DR1-PS1-z,VOID,25.858,135.456,23.969,5.658,2.17,
Gaia16aki,OBS:,57819.61490,Foundation_DR1-PS1-z,VOID,-88.730,104.138,-99.000,1.268,1.63,


In [16]:
a2 = src.FLT.unique()

In [17]:
np.sort(np.array(a2))

array(['Foundation_DR1-PS1-g', 'Foundation_DR1-PS1-i',
       'Foundation_DR1-PS1-r', 'Foundation_DR1-PS1-z'], dtype=object)

In [18]:
a1 = filtmap.filtname.unique()
# print(np.sort(a1))

In [19]:
set(a2)-set(a1)

set()

#### testing the function without using tape

In [20]:
data = ens.compute(table="source").reset_index()
meta = ens.compute(table="object").reset_index()
res = fit_lcparams(data,meta,filtmap=filtmap)

Base model info:
source:
  class      : SALT2Source
  name       : 'salt2'
  version    : T23
  phases     : [-20, .., 50] days
  wavelengths: [2000, .., 9200] Angstroms
effect (name='mw' frame='obs'):
  class           : F99Dust
  wavelength range: [909.09, 60000] Angstroms
  phase range     : [nan, nan]
parameters:
  z     = 0.0
  t0    = 0.0
  x0    = 1.0
  x1    = 0.0
  c     = 0.0
  mwebv = 0.0
0 fitting  2016W z= 0.019253 zsource= MetaData mwebv= 0.06
1 fitting  ASASSN-15bc z= 0.036715 zsource= MetaData mwebv= 0.037


/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR1-PS1-z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


2 fitting  ASASSN-15fs z= 0.02902 zsource= MetaData mwebv= 0.077
3 fitting  ASASSN-15go z= 0.018923 zsource= MetaData mwebv= 0.137
4 fitting  ASASSN-15hg z= 0.029917 zsource= MetaData mwebv= 0.042


/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR1-PS1-z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR1-PS1-z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: D

5 fitting  ASASSN-15jl z= 0.034682 zsource= MetaData mwebv= 0.014
6 fitting  ASASSN-15kx z= 0.018019 zsource= MetaData mwebv= 0.141
7 fitting  ASASSN-15lg z= 0.020151 zsource= MetaData mwebv= 0.027


...

/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR1-PS1-z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


148 fitting  SN2017coa z= 0.04 zsource= MetaData mwebv= 0.056
149 fitting  SN2017cpu z= 0.054405 zsource= MetaData mwebv= 0.024


/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


150 fitting  SN2017hn z= 0.02385 zsource= MetaData mwebv= 0.028
151 fitting  SN2017jl z= 0.016331 zsource= MetaData mwebv= 0.063
152 fitting  SN2017mf z= 0.025612 zsource= MetaData mwebv= 0.009


/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR1-PS1-z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR1-PS1-z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: D

In [26]:
res

,,,,,,success,ncall,chisq,ndof,z,z_err,t0,t0_err,x0,x0_err,...,mwebv_c_cov,mwebv_mwebv_cov,Name,Survey,ErrorType,ErrorMesg,bands,bands2,last_fitmjd,first_fitmjd
SNID,RA,DEC,REDSHIFT_HELIO,SURVEY2,MWEBV,,,,,,,,,,,,,,,,,,,,,
2016W,37.665487,42.2359685,0.019253,Foundation_DR1,0.060,1,82,30.929122,17,0.019253,0.0,57417.855715,0.099312,0.008417,0.000264,...,0.0,0.0,2016W,Foundation_DR1,None ...,None ...,gir,girz,57451.23373,57410.29056
ASASSN-15bc,61.560987,-8.8856098,0.036715,Foundation_DR1,0.037,1,71,16.753943,28,0.036715,0.0,57047.845211,0.150495,0.002583,0.000076,...,0.0,0.0,ASASSN-15bc,Foundation_DR1,None ...,None ...,girz,girz,57075.22582,57042.21498
ASASSN-15fs,284.669923,43.4688739,0.029020,Foundation_DR1,0.077,1,62,107.137489,31,0.029020,0.0,57113.232889,0.184925,0.005552,0.000166,...,0.0,0.0,ASASSN-15fs,Foundation_DR1,None ...,None ...,gir,girz,57153.60327,57110.63102
ASASSN-15go,92.876886,-16.4841129,0.018923,Foundation_DR1,0.137,1,73,18.729786,11,0.018923,0.0,57122.934673,0.811499,0.006412,0.000228,...,0.0,0.0,ASASSN-15go,Foundation_DR1,None ...,None ...,gir,girz,57316.58180,57124.25841
ASASSN-15hg,148.452399,9.1938997,0.029917,Foundation_DR1,0.042,1,73,24.972502,16,0.029917,0.0,57131.683559,0.399968,0.003183,0.000105,...,0.0,0.0,ASASSN-15hg,Foundation_DR1,None ...,None ...,gir,girz,57171.26326,57133.24702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN2017coa,132.480332,56.0990451,0.040000,Foundation_DR1,0.056,1,71,16.672700,15,0.040000,0.0,57848.696231,0.399381,0.002284,0.000081,...,0.0,0.0,SN2017coa,Foundation_DR1,None ...,None ...,girz,girz,57893.26040,57846.35159
SN2017cpu,211.968898,9.6420343,0.054405,Foundation_DR1,0.024,1,63,17.346560,19,0.054405,0.0,57847.320007,0.527189,0.002055,0.000069,...,0.0,0.0,SN2017cpu,Foundation_DR1,None ...,None ...,girz,girz,57891.31024,57847.49753
SN2017hn,196.914215,6.3376016,0.023850,Foundation_DR1,0.028,1,75,1711.283245,11,0.023850,0.0,57768.063305,0.347170,0.007760,0.000250,...,0.0,0.0,SN2017hn,Foundation_DR1,None ...,None ...,gir,girz,57823.58182,57766.64586


In [21]:
res[['x0','x1','c']]

,x0,x1,c
0,0.008417,-1.438304,0.063747
1,0.002583,0.577431,0.051361
2,0.005552,-0.030327,0.024311
3,0.006412,2.053063,0.231462
4,0.003183,-1.348221,0.015856
...,...,...,...
148,0.002284,0.882707,-0.019140
149,0.002055,0.894776,-0.074217
150,0.007760,0.720054,-0.018080
151,0.022380,0.655777,0.002265


In [22]:
# get the column names. We will use them as an input to ensemble.batch()
dfmeta = pd.DataFrame(res.dtypes.to_dict(),index=[0])

In [23]:
dfmeta

,success,ncall,chisq,ndof,z,z_err,t0,t0_err,x0,x0_err,...,mwebv_c_cov,mwebv_mwebv_cov,Name,Survey,ErrorType,ErrorMesg,bands,bands2,last_fitmjd,first_fitmjd
0,int64,int64,float64,int64,float64,float64,float64,float64,float64,float64,...,float64,float64,object,object,object,object,object,object,float64,float64


#### Now run with ensemble.batch()

In [24]:
res = ens.batch(fit_salt2_pp,"SNID","MJD","FLUXCAL","FLUXCALERR","FLT",
          "RA","DEC","REDSHIFT_HELIO","SURVEY2","MWEBV",on=["SNID","RA","DEC","REDSHIFT_HELIO","SURVEY2","MWEBV"],register_bands=True,
          meta=dfmeta,rescols=dfmeta,write_to_file=False,filtmap=filtmap)

Using generated label, result_1, for a batch result.


/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR1-PS1-z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR1-PS1-z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assi

Base model info:
source:
  class      : SALT2Source
  name       : 'salt2'
  version    : T23
  phases     : [-20, .., 50] days
  wavelengths: [2000, .., 9200] Angstroms
effect (name='mw' frame='obs'):
  class           : F99Dust
  wavelength range: [909.09, 60000] Angstroms
  phase range     : [nan, nan]
parameters:
  z     = 0.0
  t0    = 0.0
  x0    = 1.0
  x1    = 0.0
  c     = 0.0
  mwebv = 0.0
0 fitting  2016W z= 0.019253 zsource= MetaData mwebv= 0.06
Base model info:
source:
  class      : SALT2Source
  name       : 'salt2'
  version    : T23
  phases     : [-20, .., 50] days
  wavelengths: [2000, .., 9200] Angstroms
effect (name='mw' frame='obs'):
  class           : F99Dust
  wavelength range: [909.09, 60000] Angstroms
  phase range     : [nan, nan]
parameters:
  z     = 0.0
  t0    = 0.0
  x0    = 1.0
  x1    = 0.0
  c     = 0.0
  mwebv = 0.0
0 fitting  ASASSN-15bc z= 0.036715 zsource= MetaData mwebv= 0.037
Base model info:
source:
  class      : SALT2Source
  name       : 's

/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR1-PS1-z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive value

source:
  class      : SALT2Source
  name       : 'salt2'
  version    : T23
  phases     : [-20, .., 50] days
  wavelengths: [2000, .., 9200] Angstroms
effect (name='mw' frame='obs'):
  class           : F99Dust
  wavelength range: [909.09, 60000] Angstroms
  phase range     : [nan, nan]
parameters:
  z     = 0.0
  t0    = 0.0
  x0    = 1.0
  x1    = 0.0
  c     = 0.0
  mwebv = 0.0
0 fitting  ASASSN-16db z= 0.034864 zsource= MetaData mwebv= 0.023
Base model info:
source:
  class      : SALT2Source
  name       : 'salt2'
  version    : T23
  phases     : [-20, .., 50] days
  wavelengths: [2000, .., 9200] Angstroms
effect (name='mw' frame='obs'):
  class           : F99Dust
  wavelength range: [909.09, 60000] Angstroms
  phase range     : [nan, nan]
parameters:
  z     = 0.0
  t0    = 0.0
  x0    = 1.0
  x1    = 0.0
  c     = 0.0
  mwebv = 0.0
0 fitting  ASASSN-16dn z= 0.01285 zsource= MetaData mwebv= 0.033
Base model info:
source:
  class      : SALT2Source
  name       : 'salt2'
  ver

/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR1-PS1-z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR

source:
  class      : SALT2Source
  name       : 'salt2'
  version    : T23
  phases     : [-20, .., 50] days
  wavelengths: [2000, .., 9200] Angstroms
effect (name='mw' frame='obs'):
  class           : F99Dust
  wavelength range: [909.09, 60000] Angstroms
  phase range     : [nan, nan]
parameters:
  z     = 0.0
  t0    = 0.0
  x0    = 1.0
  x1    = 0.0
  c     = 0.0
  mwebv = 0.0
0 fitting  AT2017lm z= 0.030636 zsource= MetaData mwebv= 0.059
Base model info:
source:
  class      : SALT2Source
  name       : 'salt2'
  version    : T23
  phases     : [-20, .., 50] days
  wavelengths: [2000, .., 9200] Angstroms
effect (name='mw' frame='obs'):
  class           : F99Dust
  wavelength range: [909.09, 60000] Angstroms
  phase range     : [nan, nan]
parameters:
  z     = 0.0
  t0    = 0.0
  x0    = 1.0
  x1    = 0.0
  c     = 0.0
  mwebv = 0.0
0 fitting  AT2017ns z= 0.028766 zsource= MetaData mwebv= 0.094
Base model info:
source:
  class      : SALT2Source
  name       : 'salt2'
  version 

/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR1-PS1-z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive value

source:
  class      : SALT2Source
  name       : 'salt2'
  version    : T23
  phases     : [-20, .., 50] days
  wavelengths: [2000, .., 9200] Angstroms
effect (name='mw' frame='obs'):
  class           : F99Dust
  wavelength range: [909.09, 60000] Angstroms
  phase range     : [nan, nan]
parameters:
  z     = 0.0
  t0    = 0.0
  x0    = 1.0
  x1    = 0.0
  c     = 0.0
  mwebv = 0.0
0 fitting  PS15bjg z= 0.068888 zsource= MetaData mwebv= 0.079
Base model info:
source:
  class      : SALT2Source
  name       : 'salt2'
  version    : T23
  phases     : [-20, .., 50] days
  wavelengths: [2000, .., 9200] Angstroms
effect (name='mw' frame='obs'):
  class           : F99Dust
  wavelength range: [909.09, 60000] Angstroms
  phase range     : [nan, nan]
parameters:
  z     = 0.0
  t0    = 0.0
  x0    = 1.0
  x1    = 0.0
  c     = 0.0
  mwebv = 0.0
0 fitting  PS15brr z= 0.051804 zsource= MetaData mwebv= 0.028
Base model info:
source:
  class      : SALT2Source
  name       : 'salt2'
  version   

/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR1-PS1-z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/Users/mi/anaconda3/envs/tape_env/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: Foundation_DR

In [25]:
res[['x0','x1','c','bands']]

,,,,,,x0,x1,c,bands
SNID,RA,DEC,REDSHIFT_HELIO,SURVEY2,MWEBV,,,,
2016W,37.665487,42.2359685,0.019253,Foundation_DR1,0.060,0.008417,-1.438304,0.063747,gir
ASASSN-15bc,61.560987,-8.8856098,0.036715,Foundation_DR1,0.037,0.002583,0.577431,0.051361,girz
ASASSN-15fs,284.669923,43.4688739,0.029020,Foundation_DR1,0.077,0.005552,-0.030327,0.024311,gir
ASASSN-15go,92.876886,-16.4841129,0.018923,Foundation_DR1,0.137,0.006412,2.053063,0.231462,gir
ASASSN-15hg,148.452399,9.1938997,0.029917,Foundation_DR1,0.042,0.003183,-1.348221,0.015856,gir
...,...,...,...,...,...,...,...,...,...
SN2017coa,132.480332,56.0990451,0.040000,Foundation_DR1,0.056,0.002284,0.882707,-0.019140,girz
SN2017cpu,211.968898,9.6420343,0.054405,Foundation_DR1,0.024,0.002055,0.894776,-0.074217,girz
SN2017hn,196.914215,6.3376016,0.023850,Foundation_DR1,0.028,0.007760,0.720054,-0.018080,gir
